In [ ]:
# imports 
import os
import sys 

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import scipy.interpolate as interp

sys.path.append('/l/cmich/DAFI')
from dafi.random_field import foam_utilities as foam

# constants
C_mu = 0.09
tinit = 0
SYMTENSORDIM = 6
TENSORSQRTDIM = 3
TENSORDIM = 9
DEVSYMTENSORDIM = 5
DEVSYMTENSOR_INDEX = [0,1,2,4,5]

# Case

In [ ]:
tfinal = 100

foam_dir = 'foam_primal'
results_dir = 'results'
data_dir = 'data'
save_dir = 'figures'

nscalar_invariants = 1
nbasis_tensors = 1
mesh_shape = [30, 100]

cmap = 'Spectral' 

# plotting functions

In [ ]:
def rs(x):
    return x.reshape(mesh_shape)


def get_lim(x):
    return np.max(np.abs(x))

# load data

In [ ]:
# coordinates
x = np.loadtxt(os.path.join(data_dir, 'x'))
y = np.loadtxt(os.path.join(data_dir, 'y'))

# cost
J = []
for i in range(tfinal+1):
    iJ = np.loadtxt(os.path.join(results_dir, f'J.{i}'))
    J.append(iJ.tolist())

# initial
g_0 = np.loadtxt(os.path.join(results_dir, f'g.{tinit}'))
g_0 = np.expand_dims(g_0, -1)
Ux_0 = np.loadtxt(os.path.join(results_dir, f'U.flow_0.{tinit}'))[:, 0]
Uy_0 = np.loadtxt(os.path.join(results_dir, f'U.flow_0.{tinit}'))[:, 1]

# final
g_1 = np.loadtxt(os.path.join(results_dir, f'g.{tfinal}'))
g_1 = np.expand_dims(g_1, -1)
Ux_1 = np.loadtxt(os.path.join(results_dir, f'U.flow_0.{tfinal}'))[:,0]
Uy_1 = np.loadtxt(os.path.join(results_dir, f'U.flow_0.{tfinal}'))[:,1]

# truth
Ux_t = np.loadtxt(os.path.join(data_dir, 'UxFullField')) 
Uy_t = np.loadtxt(os.path.join(data_dir, 'UyFullField')) 

print('Done')

# Results

### Cost

In [ ]:
fig, ax = plt.subplots()
ax.plot(J[:tfinal+1], '-')
ax.set_xlabel(r'iteration')
ax.set_ylabel(r'$J$')
ax.set_title(r'Cost $J$')
ax.set_ylim(0., np.max(J[:tfinal+1]))
ax.set_xlim(0., tfinal)
    
fig, ax = plt.subplots()
ax.semilogy(J[:tfinal+1], '-')
ax.set_xlabel(r'iteration')
ax.set_ylabel(r'$J$')
ax.set_title(r'Cost $J$')
limlow = 10**np.floor(np.log10(np.min(J[:tfinal+1])))
ax.set_ylim(limlow, np.max(J[:tfinal+1]))
ax.set_xlim(0., tfinal);
   

### functions $g^{(i)}$

In [ ]:
for j in range(g_0.shape[1]):

    fig, axs = plt.subplots(2,1, figsize=(10,10))
    
    cs = axs[0].pcolor(rs(x), rs(y), rs(g_0[:,j]), cmap=cmap, shading='auto', norm=None)
    cbar = fig.colorbar(cs, ax=axs[0], shrink=0.7)

    cs = axs[1].pcolor(rs(x), rs(y), rs(g_1[:,j]), cmap=cmap, shading='auto', norm=None)
    cbar = fig.colorbar(cs, ax=axs[1], shrink=0.7)

    for i in range(2):
        plt.setp(axs[i].get_xticklabels(), visible=False)
        plt.setp(axs[i].get_yticklabels(), visible=False)
        axs[i].tick_params(axis='both', which='both', length=0)
        axs[i].set_aspect('equal', 'box')
    axs[0].set_title('g-functions')
    axs[0].set_ylabel('initial')
    axs[1].set_ylabel('final')
    axs[0].set_title(f'g_{j+1}')


### Velocity

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,15))

divnorm = mcolors.TwoSlopeNorm(vcenter=0.0)
cs = axs[0].pcolor(rs(x), rs(y), rs(Ux_0), cmap=cmap, shading='auto', norm=divnorm)
cbar = fig.colorbar(cs, ax=axs[0], shrink=0.7)

divnorm = mcolors.TwoSlopeNorm(vcenter=0.0)
cs = axs[1].pcolor(rs(x), rs(y), rs(Ux_1), cmap=cmap, shading='auto', norm=divnorm)
cbar = fig.colorbar(cs, ax=axs[1], shrink=0.7)

divnorm = mcolors.TwoSlopeNorm(vcenter=0.0)
cs = axs[2].pcolor(rs(x), rs(y), rs(Ux_t), cmap=cmap, shading='auto', norm=divnorm)
cbar = fig.colorbar(cs, ax=axs[2], shrink=0.7)

for i in range(3):
    plt.setp(axs[i].get_xticklabels(), visible=False)
    plt.setp(axs[i].get_yticklabels(), visible=False)
    axs[i].tick_params(axis='both', which='both', length=0)
    axs[i].set_aspect('equal', 'box')
axs[0].set_title('Ux-velocity')
axs[0].set_ylabel('initial')
axs[1].set_ylabel('final')
axs[2].set_ylabel('truth')

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(10,15))

divnorm = mcolors.TwoSlopeNorm(vcenter=0.0)
cs = axs[0].pcolor(rs(x), rs(y), rs(Uy_0), cmap=cmap, shading='auto', norm=divnorm)
cbar = fig.colorbar(cs, ax=axs[0], shrink=0.7)

divnorm = mcolors.TwoSlopeNorm(vcenter=0.0)
cs = axs[1].pcolor(rs(x), rs(y), rs(Uy_1), cmap=cmap, shading='auto', norm=divnorm)
cbar = fig.colorbar(cs, ax=axs[1], shrink=0.7)

divnorm = mcolors.TwoSlopeNorm(vcenter=0.0)
cs = axs[2].pcolor(rs(x), rs(y), rs(Uy_t), cmap=cmap, shading='auto', norm=divnorm)
cbar = fig.colorbar(cs, ax=axs[2], shrink=0.7)

for i in range(3):
    plt.setp(axs[i].get_xticklabels(), visible=False)
    plt.setp(axs[i].get_yticklabels(), visible=False)
    axs[i].tick_params(axis='both', which='both', length=0)
    axs[i].set_aspect('equal', 'box')
axs[0].set_title('Uy-velocity')
axs[0].set_ylabel('initial')
axs[1].set_ylabel('final')
axs[2].set_ylabel('truth')